In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import date

In [2]:
url = 'https://raw.githubusercontent.com/datasets/covid-19/main/data/countries-aggregated.csv'
df = pd.read_csv(url, error_bad_lines=False)

df['Date'] = pd.to_datetime(df['Date'])
df['Country'] = df['Country'].astype('category')

df.head()

,Date,Country,Confirmed,Recovered,Deaths
0,2020-01-22,Afghanistan,0,0,0
1,2020-01-23,Afghanistan,0,0,0
2,2020-01-24,Afghanistan,0,0,0
3,2020-01-25,Afghanistan,0,0,0
4,2020-01-26,Afghanistan,0,0,0


In [3]:
url = 'https://raw.githubusercontent.com/lukes/ISO-3166-Countries-with-Regional-Codes/master/all/all.csv'
country_codes = pd.read_csv(url, error_bad_lines=False)
country_codes.head()

,name,alpha-2,alpha-3,country-code,iso_3166-2,region,sub-region,intermediate-region,region-code,sub-region-code,intermediate-region-code
0,Afghanistan,AF,AFG,4,ISO 3166-2:AF,Asia,Southern Asia,NaN,142.0,34.0,NaN
1,Åland Islands,AX,ALA,248,ISO 3166-2:AX,Europe,Northern Europe,NaN,150.0,154.0,NaN
2,Albania,AL,ALB,8,ISO 3166-2:AL,Europe,Southern Europe,NaN,150.0,39.0,NaN
3,Algeria,DZ,DZA,12,ISO 3166-2:DZ,Africa,Northern Africa,NaN,2.0,15.0,NaN
4,American Samoa,AS,ASM,16,ISO 3166-2:AS,Oceania,Polynesia,NaN,9.0,61.0,NaN


In [4]:
event_dictionary = dict(zip(country_codes.iloc[:,0].values, country_codes.iloc[:,2].values))

In [5]:
confirmed = df[['Country', 'Confirmed']].groupby(['Country']).max()
confirmed = confirmed.reset_index(level=['Country'])
confirmed['country_iso'] = confirmed['Country'].map(event_dictionary)
confirmed

,Country,Confirmed,country_iso
0,Afghanistan,56454,AFG
1,Albania,125157,ALB
2,Algeria,117192,DZA
3,Andorra,12010,AND
4,Angola,22311,AGO
...,...,...,...
187,Vietnam,2603,NaN
188,West Bank and Gaza,242353,NaN
189,Yemen,4357,YEM
190,Zambia,88418,ZMB


In [18]:
confirmed.loc[confirmed['Country'].str.contains('Bolivia'), 'country_iso'] = 'BOL'
confirmed.loc[confirmed['Country'].str.contains('Burma'), 'country_iso'] = 'MMR'
confirmed.loc[confirmed['Country'].str.contains('Brunei'), 'country_iso'] = 'BRN'
confirmed.loc[confirmed['Country'].str.contains("Congo (Brazzaville)"), 'country_iso'] = 'COG'
confirmed.loc[confirmed['Country'].str.contains("Congo (Kinshasa)"), 'country_iso'] = 'COD'
confirmed.loc[confirmed['Country'].str.contains("Cote d'Ivoire"), 'country_iso'] = 'CIV'
confirmed.loc[confirmed['Country'].str.contains('Iran'), 'country_iso'] = 'IRN'
confirmed.loc[confirmed['Country'].str.contains('Korea, South'), 'country_iso'] = 'KOR'
confirmed.loc[confirmed['Country'].str.contains('Russia'), 'country_iso'] = 'RUS'
confirmed.loc[confirmed['Country'].str.contains('Syria'), 'country_iso'] = 'SYR'
confirmed.loc[confirmed['Country'].str.contains('US'), 'country_iso'] = 'USA'
confirmed.loc[confirmed['Country'].str.contains('United Kingdom'), 'country_iso'] = 'GBR'
confirmed.loc[confirmed['Country'].str.contains('Venezuela'), 'country_iso'] = 'VEN'
confirmed.loc[confirmed['Country'].str.contains('Vietnam'), 'country_iso'] = 'VNM'

In [19]:
for i in range(confirmed.shape[0]):
    if pd.isna(confirmed.iloc[i,2]):
        print(confirmed.iloc[i,0])

Congo (Brazzaville)
Congo (Kinshasa)
Diamond Princess
Kosovo
Laos
MS Zaandam
Micronesia
Moldova
Taiwan*
Tanzania
West Bank and Gaza


In [20]:
import plotly.graph_objects as go

fig = go.Figure(data=go.Choropleth(
    locations=confirmed['country_iso'],
    z=confirmed['Confirmed'],
    text=confirmed['Country'],
    colorscale='Blues',
    autocolorscale=False,
    reversescale=False,
    marker_line_color='darkgray',
    marker_line_width=0.5,
    # colorbar_tickprefix = '$',
    colorbar_title='Sum of Confirmed<br>Covid-19 cases',
))

fig.update_layout(
    title_text='Confirmed Cases '+ str(df['Date'].max()),
    geo=dict(
        showframe=False,
        showcoastlines=False,
        projection_type='equirectangular'
    ),
   )

fig.show()